<h1>ENB30 - Revisiting FFNs with K-fold cross validation</h1>
<h2> activation functions </h2>

In [2]:
import tensorflow as tf
import keras
from keras import models
from keras import layers

import numpy as np
import os
import csv

import pickle
import time as tm

import matplotlib.pyplot as plt
import pre_process as pp

Using TensorFlow backend.


<h2> Loading the datasets</h2>

In [3]:
with open('train_data.txt', 'rb') as fp:
    train_data = pickle.load(fp)
    
with open('train_labels.txt', 'rb') as fp:
    train_labels = pickle.load(fp)
    
with open('test_data.txt', 'rb') as fp:
    test_data = pickle.load(fp)
    
with open('test_labels.txt', 'rb') as fp:
    test_labels = pickle.load(fp)

<h2>k-fold params</h2>

In [4]:
k = 4
num_val_samples = len(train_data)
print(num_val_samples)
val_split_size = num_val_samples/k
val_split_size = int(val_split_size)
print(val_split_size)
type(val_split_size)

72
18


int

<h2>Experiment1: a single layer</h2>

In [21]:
net1 = models.Sequential()
net1.add(layers.Dense((480), input_shape = (480,6)))
net1.add(layers.Flatten())
net1.add(layers.Dense(240,activation='relu'))
net1.add(layers.Dense(240,activation='relu'))
net1.add(layers.Dense(240,activation='relu'))
net1.add(layers.Dense(7, activation = 'softmax'))
net1.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_31 (Dense)             (None, 480, 480)          3360      
_________________________________________________________________
flatten_7 (Flatten)          (None, 230400)            0         
_________________________________________________________________
dense_32 (Dense)             (None, 240)               55296240  
_________________________________________________________________
dense_33 (Dense)             (None, 240)               57840     
_________________________________________________________________
dense_34 (Dense)             (None, 240)               57840     
_________________________________________________________________
dense_35 (Dense)             (None, 7)                 1687      
Total params: 55,416,967
Trainable params: 55,416,967
Non-trainable params: 0
__________________________________________

In [22]:
net1.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
net1.save_weights('net1.h5')  # each split will start with same weights so can directly compare results

In [23]:

histories1 = []
results_list1 = []
index = 0
fold = val_split_size



start = tm.time()

for i in range (k):
    print(i)
    val_fold = train_data[index:fold]
    train_fold = np.delete(train_data, np.s_[index:fold],0)

    val_tgt = train_labels[index:fold]
    train_tgt = np.delete(train_labels, np.s_[index:fold])
    
    
    net1.load_weights('net1.h5')
    history = net1.fit(train_fold, train_tgt, epochs = 25, validation_data = (val_fold, val_tgt))

    histories1.append(history)
    
    
    results = net1.evaluate(test_data, test_labels)
    results_list1.append(results)
    print(results)
    
    index = index + val_split_size
    fold = fold + val_split_size
    
finish = tm.time()
duration1 = finish - start

0
Train on 54 samples, validate on 18 samples
Epoch 1/25
54/54 [==============================] - 2s 31ms/step - loss: 4.9476 - accuracy: 0.1481 - val_loss: 5.3270 - val_accuracy: 0.1111
Epoch 2/25
54/54 [==============================] - 1s 26ms/step - loss: 4.4238 - accuracy: 0.3519 - val_loss: 3.0358 - val_accuracy: 0.2222
Epoch 3/25
54/54 [==============================] - 1s 26ms/step - loss: 3.4260 - accuracy: 0.1667 - val_loss: 2.1944 - val_accuracy: 0.1667
Epoch 4/25
54/54 [==============================] - 1s 27ms/step - loss: 2.1677 - accuracy: 0.2222 - val_loss: 2.0787 - val_accuracy: 0.1667
Epoch 5/25
54/54 [==============================] - 1s 26ms/step - loss: 1.9482 - accuracy: 0.2963 - val_loss: 1.4264 - val_accuracy: 0.5000
Epoch 6/25
54/54 [==============================] - 1s 26ms/step - loss: 1.6146 - accuracy: 0.4444 - val_loss: 1.5089 - val_accuracy: 0.3333
Epoch 7/25
54/54 [==============================] - 1s 27ms/step - loss: 1.3421 - accuracy: 0.4630 - val_los

54/54 [==============================] - 1s 27ms/step - loss: 1.7081 - accuracy: 0.5000 - val_loss: 1.7325 - val_accuracy: 0.2222
Epoch 7/25
54/54 [==============================] - 1s 26ms/step - loss: 1.2870 - accuracy: 0.4630 - val_loss: 1.4888 - val_accuracy: 0.3889
Epoch 8/25
54/54 [==============================] - 1s 27ms/step - loss: 1.1194 - accuracy: 0.6111 - val_loss: 1.4641 - val_accuracy: 0.4444
Epoch 9/25
54/54 [==============================] - 1s 26ms/step - loss: 1.0533 - accuracy: 0.5741 - val_loss: 1.4653 - val_accuracy: 0.2778
Epoch 10/25
54/54 [==============================] - 1s 26ms/step - loss: 0.8883 - accuracy: 0.5556 - val_loss: 1.1089 - val_accuracy: 0.4444
Epoch 11/25
54/54 [==============================] - 1s 26ms/step - loss: 0.7767 - accuracy: 0.6111 - val_loss: 1.0373 - val_accuracy: 0.3889
Epoch 12/25
54/54 [==============================] - 1s 26ms/step - loss: 0.6087 - accuracy: 0.7963 - val_loss: 0.8525 - val_accuracy: 0.6667
Epoch 13/25
54/54 [==

In [24]:
avg = np.mean(results_list1, axis=0)
std_dev = np.std(results_list1, axis=0)
print('Average loss and accuracy scores were: ', avg)
print('Standard deviation for loss & accuracy: ', std_dev)
print('Duration in seconds: ', int(duration1))

Average loss and accuracy scores were:  [0.53611019 0.75000001]
Standard deviation for loss & accuracy:  [0.15338172 0.08660254]
Duration in seconds:  145


<h2> Expriment2:</h2>

In [5]:
net2 = models.Sequential()
net2.add(layers.Dense((480), input_shape = (480,6)))
net2.add(layers.Flatten())
net2.add(layers.Dense(240,activation='softmax'))
net2.add(layers.Dense(240,activation='softmax'))
net2.add(layers.Dense(240,activation='softmax'))
net2.add(layers.Dense(7, activation = 'softmax'))
net2.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 480, 480)          3360      
_________________________________________________________________
flatten_1 (Flatten)          (None, 230400)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 240)               55296240  
_________________________________________________________________
dense_3 (Dense)              (None, 240)               57840     
_________________________________________________________________
dense_4 (Dense)              (None, 240)               57840     
_________________________________________________________________
dense_5 (Dense)              (None, 7)                 1687      
Total params: 55,416,967
Trainable para

In [8]:
net2.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
net2.save_weights('net2.h5')  # each split will start with same weights so can directly compare results

histories2 = []
results_list2 = []
index = 0
fold = val_split_size



start = tm.time()

for i in range (k):
    print(i)
    val_fold = train_data[index:fold]
    train_fold = np.delete(train_data, np.s_[index:fold],0)

    val_tgt = train_labels[index:fold]
    train_tgt = np.delete(train_labels, np.s_[index:fold])
    
    
    net2.load_weights('net2.h5')
    history = net2.fit(train_fold, train_tgt, epochs = 25, validation_data = (val_fold, val_tgt))

    histories2.append(history)
    
    
    results = net2.evaluate(test_data, test_labels)
    results_list2.append(results)
    print(results)
    
    index = index + val_split_size
    fold = fold + val_split_size

finish = tm.time()
duration2 = finish - start

0
Train on 54 samples, validate on 18 samples
Epoch 1/25
54/54 [==============================] - 2s 28ms/step - loss: 1.9357 - accuracy: 0.1667 - val_loss: 1.9231 - val_accuracy: 0.2222
Epoch 2/25
54/54 [==============================] - 1s 26ms/step - loss: 1.9351 - accuracy: 0.1667 - val_loss: 1.9223 - val_accuracy: 0.2222
Epoch 3/25
54/54 [==============================] - 1s 26ms/step - loss: 1.9344 - accuracy: 0.1667 - val_loss: 1.9214 - val_accuracy: 0.2222
Epoch 4/25
54/54 [==============================] - 1s 26ms/step - loss: 1.9338 - accuracy: 0.1667 - val_loss: 1.9206 - val_accuracy: 0.2222
Epoch 5/25
54/54 [==============================] - 1s 27ms/step - loss: 1.9334 - accuracy: 0.1667 - val_loss: 1.9198 - val_accuracy: 0.2222
Epoch 6/25
54/54 [==============================] - 1s 26ms/step - loss: 1.9330 - accuracy: 0.1667 - val_loss: 1.9189 - val_accuracy: 0.2222
Epoch 7/25
54/54 [==============================] - 1s 26ms/step - loss: 1.9324 - accuracy: 0.1667 - val_los

54/54 [==============================] - 1s 27ms/step - loss: 1.9221 - accuracy: 0.2037 - val_loss: 1.9494 - val_accuracy: 0.1111
Epoch 7/25
54/54 [==============================] - 1s 26ms/step - loss: 1.9209 - accuracy: 0.2037 - val_loss: 1.9501 - val_accuracy: 0.1111
Epoch 8/25
54/54 [==============================] - 1s 27ms/step - loss: 1.9194 - accuracy: 0.2037 - val_loss: 1.9507 - val_accuracy: 0.1111
Epoch 9/25
54/54 [==============================] - 1s 27ms/step - loss: 1.9181 - accuracy: 0.2037 - val_loss: 1.9514 - val_accuracy: 0.1111
Epoch 10/25
54/54 [==============================] - 1s 27ms/step - loss: 1.9167 - accuracy: 0.2037 - val_loss: 1.9521 - val_accuracy: 0.1111
Epoch 11/25
54/54 [==============================] - 1s 26ms/step - loss: 1.9152 - accuracy: 0.2037 - val_loss: 1.9528 - val_accuracy: 0.1111
Epoch 12/25
54/54 [==============================] - 1s 26ms/step - loss: 1.9139 - accuracy: 0.2037 - val_loss: 1.9535 - val_accuracy: 0.1111
Epoch 13/25
54/54 [==

In [9]:
avg = np.mean(results_list2, axis=0)
std_dev = np.std(results_list2, axis=0)
print('Average loss and accuracy scores were: ', avg)
print('Standard deviation for loss & accuracy: ', std_dev)
print('Duration in seconds: ', int(duration2))

Average loss and accuracy scores were:  [1.92013758 0.16666667]
Standard deviation for loss & accuracy:  [0.00254598 0.03333333]
Duration in seconds:  145


<h2> Expriment3:</h2>

In [14]:
net3 = models.Sequential()
net3.add(layers.Dense((480), input_shape = (480,6)))
net3.add(layers.Flatten())
net3.add(layers.Dense(240,activation='sigmoid'))
net3.add(layers.Dense(240,activation='sigmoid'))
net3.add(layers.Dense(240,activation='sigmoid'))
net3.add(layers.Dense(7, activation = 'softmax'))
net3.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 480, 480)          3360      
_________________________________________________________________
flatten_4 (Flatten)          (None, 230400)            0         
_________________________________________________________________
dense_17 (Dense)             (None, 240)               55296240  
_________________________________________________________________
dense_18 (Dense)             (None, 240)               57840     
_________________________________________________________________
dense_19 (Dense)             (None, 240)               57840     
_________________________________________________________________
dense_20 (Dense)             (None, 7)                 1687      
Total params: 55,416,967
Trainable params: 55,416,967
Non-trainable params: 0
__________________________________________

In [15]:
net3.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
net3.save_weights('net3.h5')  # each split will start with same weights so can directly compare results

histories3 = []
results_list3 = []
index = 0
fold = val_split_size



start = tm.time()

for i in range (k):
    print(i)
    val_fold = train_data[index:fold]
    train_fold = np.delete(train_data, np.s_[index:fold],0)

    val_tgt = train_labels[index:fold]
    train_tgt = np.delete(train_labels, np.s_[index:fold])
    
    
    net3.load_weights('net3.h5')
    history = net3.fit(train_fold, train_tgt, epochs = 58, validation_data = (val_fold, val_tgt))

    histories3.append(history)
    
    
    results = net3.evaluate(test_data, test_labels)
    results_list3.append(results)
    print(results)
    
    index = index + val_split_size
    fold = fold + val_split_size

finish = tm.time()
duration3 = finish - start

0
Train on 54 samples, validate on 18 samples
Epoch 1/58
54/54 [==============================] - 2s 30ms/step - loss: 2.1651 - accuracy: 0.1111 - val_loss: 1.8593 - val_accuracy: 0.2222
Epoch 2/58
54/54 [==============================] - 1s 26ms/step - loss: 1.9244 - accuracy: 0.2037 - val_loss: 1.9162 - val_accuracy: 0.1111
Epoch 3/58
54/54 [==============================] - 1s 26ms/step - loss: 1.9129 - accuracy: 0.1852 - val_loss: 1.8921 - val_accuracy: 0.1111
Epoch 4/58
54/54 [==============================] - 1s 27ms/step - loss: 1.9164 - accuracy: 0.1852 - val_loss: 1.8563 - val_accuracy: 0.1111
Epoch 5/58
54/54 [==============================] - 1s 26ms/step - loss: 1.8992 - accuracy: 0.1296 - val_loss: 1.8263 - val_accuracy: 0.1667
Epoch 6/58
54/54 [==============================] - 1s 26ms/step - loss: 1.8838 - accuracy: 0.1667 - val_loss: 1.7993 - val_accuracy: 0.1667
Epoch 7/58
54/54 [==============================] - 1s 26ms/step - loss: 1.8795 - accuracy: 0.2037 - val_los

15/15 [==============================] - 0s 1ms/step
[0.46427738666534424, 0.800000011920929]
1
Train on 54 samples, validate on 18 samples
Epoch 1/58
54/54 [==============================] - 1s 26ms/step - loss: 2.0857 - accuracy: 0.1481 - val_loss: 1.8593 - val_accuracy: 0.2222
Epoch 2/58
54/54 [==============================] - 1s 26ms/step - loss: 1.9326 - accuracy: 0.1667 - val_loss: 1.8493 - val_accuracy: 0.1667
Epoch 3/58
54/54 [==============================] - 1s 27ms/step - loss: 1.9177 - accuracy: 0.2222 - val_loss: 1.8889 - val_accuracy: 0.2222
Epoch 4/58
54/54 [==============================] - 1s 27ms/step - loss: 1.9437 - accuracy: 0.2222 - val_loss: 1.9079 - val_accuracy: 0.1667
Epoch 5/58
54/54 [==============================] - 1s 27ms/step - loss: 1.9408 - accuracy: 0.2037 - val_loss: 1.8744 - val_accuracy: 0.1667
Epoch 6/58
54/54 [==============================] - 1s 27ms/step - loss: 1.9208 - accuracy: 0.1852 - val_loss: 1.8317 - val_accuracy: 0.1667
Epoch 7/58
54/

Epoch 57/58
54/54 [==============================] - 1s 27ms/step - loss: 0.9845 - accuracy: 0.6481 - val_loss: 0.7919 - val_accuracy: 0.8889
Epoch 58/58
15/15 [==============================] - 0s 1ms/step
[0.936518669128418, 0.8666666746139526]
2
Train on 54 samples, validate on 18 samples
Epoch 1/58
54/54 [==============================] - 1s 26ms/step - loss: 2.0396 - accuracy: 0.1296 - val_loss: 1.9788 - val_accuracy: 0.1111
Epoch 2/58
54/54 [==============================] - 1s 27ms/step - loss: 1.8682 - accuracy: 0.2222 - val_loss: 2.0854 - val_accuracy: 0.1111
Epoch 3/58
54/54 [==============================] - 1s 27ms/step - loss: 1.8164 - accuracy: 0.2593 - val_loss: 2.2585 - val_accuracy: 0.1111
Epoch 4/58
54/54 [==============================] - 1s 26ms/step - loss: 1.8357 - accuracy: 0.2037 - val_loss: 2.3707 - val_accuracy: 0.1111
Epoch 5/58
54/54 [==============================] - 1s 27ms/step - loss: 1.8591 - accuracy: 0.2593 - val_loss: 2.3997 - val_accuracy: 0.2222
Ep

Epoch 56/58
54/54 [==============================] - 1s 27ms/step - loss: 0.6841 - accuracy: 0.7407 - val_loss: 1.1507 - val_accuracy: 0.3333
Epoch 57/58
54/54 [==============================] - 1s 26ms/step - loss: 0.6731 - accuracy: 0.7407 - val_loss: 1.1261 - val_accuracy: 0.3889
Epoch 58/58
15/15 [==============================] - 0s 1ms/step
[0.8480743765830994, 0.6666666865348816]
3
Train on 54 samples, validate on 18 samples
Epoch 1/58
54/54 [==============================] - 1s 26ms/step - loss: 1.9415 - accuracy: 0.2222 - val_loss: 2.1238 - val_accuracy: 0.0000e+00
Epoch 2/58
54/54 [==============================] - 1s 26ms/step - loss: 1.8530 - accuracy: 0.2222 - val_loss: 2.0367 - val_accuracy: 0.0000e+00
Epoch 3/58
54/54 [==============================] - 1s 27ms/step - loss: 1.8369 - accuracy: 0.2222 - val_loss: 1.9764 - val_accuracy: 0.1667
Epoch 4/58
54/54 [==============================] - 1s 27ms/step - loss: 1.8423 - accuracy: 0.1667 - val_loss: 1.9738 - val_accuracy:

Epoch 55/58
54/54 [==============================] - 1s 26ms/step - loss: 0.7029 - accuracy: 0.7407 - val_loss: 1.3279 - val_accuracy: 0.3889
Epoch 56/58
54/54 [==============================] - 1s 27ms/step - loss: 0.6942 - accuracy: 0.7407 - val_loss: 1.3487 - val_accuracy: 0.3889
Epoch 57/58
54/54 [==============================] - 1s 26ms/step - loss: 0.6858 - accuracy: 0.7407 - val_loss: 1.4104 - val_accuracy: 0.3333
Epoch 58/58
15/15 [==============================] - 0s 2ms/step
[0.7778826355934143, 0.6000000238418579]


In [16]:
avg = np.mean(results_list3, axis=0)
std_dev = np.std(results_list3, axis=0)
print('Average loss and accuracy scores were: ', avg)
print('Standard deviation for loss & accuracy: ', std_dev)
print('Duration in seconds: ', int(duration3))

Average loss and accuracy scores were:  [0.75668827 0.73333335]
Standard deviation for loss & accuracy:  [0.17793518 0.10540925]
Duration in seconds:  335


<h2> Expriment4:</h2>

In [25]:
net4 = models.Sequential()
net4.add(layers.Dense((480), input_shape = (480,6)))
net4.add(layers.Flatten())
net4.add(layers.Dense(240,activation='tanh'))
net4.add(layers.Dense(240,activation='tanh'))
net4.add(layers.Dense(240,activation='tanh'))
net4.add(layers.Dense(7, activation = 'softmax'))
net4.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_36 (Dense)             (None, 480, 480)          3360      
_________________________________________________________________
flatten_8 (Flatten)          (None, 230400)            0         
_________________________________________________________________
dense_37 (Dense)             (None, 240)               55296240  
_________________________________________________________________
dense_38 (Dense)             (None, 240)               57840     
_________________________________________________________________
dense_39 (Dense)             (None, 240)               57840     
_________________________________________________________________
dense_40 (Dense)             (None, 7)                 1687      
Total params: 55,416,967
Trainable params: 55,416,967
Non-trainable params: 0
__________________________________________

In [26]:
net4.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
net4.save_weights('net4.h5')  # each split will start with same weights so can directly compare results

histories4 = []
results_list4 = []
index = 0
fold = val_split_size



start = tm.time()

for i in range (k):
    print(i)
    val_fold = train_data[index:fold]
    train_fold = np.delete(train_data, np.s_[index:fold],0)

    val_tgt = train_labels[index:fold]
    train_tgt = np.delete(train_labels, np.s_[index:fold])
    
    
    net4.load_weights('net4.h5')
    history = net4.fit(train_fold, train_tgt, epochs = 50, validation_data = (val_fold, val_tgt))

    histories4.append(history)
    
    
    results = net4.evaluate(test_data, test_labels)
    results_list4.append(results)
    print(results)
    
    index = index + val_split_size
    fold = fold + val_split_size

finish = tm.time()
duration4 = finish - start

0
Train on 54 samples, validate on 18 samples
Epoch 1/50
54/54 [==============================] - 2s 31ms/step - loss: 2.5388 - accuracy: 0.1111 - val_loss: 1.6916 - val_accuracy: 0.3333
Epoch 2/50
54/54 [==============================] - 1s 26ms/step - loss: 2.2356 - accuracy: 0.2778 - val_loss: 1.7518 - val_accuracy: 0.3333
Epoch 3/50
54/54 [==============================] - 1s 27ms/step - loss: 2.0307 - accuracy: 0.3148 - val_loss: 1.5607 - val_accuracy: 0.3333
Epoch 4/50
54/54 [==============================] - 1s 27ms/step - loss: 1.7999 - accuracy: 0.2037 - val_loss: 1.7186 - val_accuracy: 0.3333
Epoch 5/50
54/54 [==============================] - 1s 26ms/step - loss: 1.7402 - accuracy: 0.3333 - val_loss: 1.6333 - val_accuracy: 0.3333
Epoch 6/50
54/54 [==============================] - 1s 27ms/step - loss: 1.6014 - accuracy: 0.3333 - val_loss: 1.4744 - val_accuracy: 0.4444
Epoch 7/50
54/54 [==============================] - 1s 26ms/step - loss: 1.4901 - accuracy: 0.2778 - val_los

54/54 [==============================] - 1s 26ms/step - loss: 1.6431 - accuracy: 0.4444 - val_loss: 1.3032 - val_accuracy: 0.3889
Epoch 8/50
54/54 [==============================] - 1s 26ms/step - loss: 1.4976 - accuracy: 0.3889 - val_loss: 1.3424 - val_accuracy: 0.3889
Epoch 9/50
54/54 [==============================] - 1s 26ms/step - loss: 1.4571 - accuracy: 0.4074 - val_loss: 1.2103 - val_accuracy: 0.4444
Epoch 10/50
54/54 [==============================] - 1s 27ms/step - loss: 1.3953 - accuracy: 0.5000 - val_loss: 1.1306 - val_accuracy: 0.6111
Epoch 11/50
54/54 [==============================] - 1s 27ms/step - loss: 1.3242 - accuracy: 0.5556 - val_loss: 1.1540 - val_accuracy: 0.6111
Epoch 12/50
54/54 [==============================] - 1s 26ms/step - loss: 1.3033 - accuracy: 0.5185 - val_loss: 1.0324 - val_accuracy: 0.7222
Epoch 13/50
54/54 [==============================] - 1s 26ms/step - loss: 1.2014 - accuracy: 0.5556 - val_loss: 1.0136 - val_accuracy: 0.5556
Epoch 14/50
54/54 [=

Epoch 14/50
54/54 [==============================] - 1s 27ms/step - loss: 1.5190 - accuracy: 0.4630 - val_loss: 1.9986 - val_accuracy: 0.1667
Epoch 15/50
54/54 [==============================] - 1s 27ms/step - loss: 1.5591 - accuracy: 0.4259 - val_loss: 1.8253 - val_accuracy: 0.2222
Epoch 16/50
54/54 [==============================] - 1s 28ms/step - loss: 1.5939 - accuracy: 0.3519 - val_loss: 1.7588 - val_accuracy: 0.3889
Epoch 17/50
54/54 [==============================] - 1s 27ms/step - loss: 1.4913 - accuracy: 0.3889 - val_loss: 1.7458 - val_accuracy: 0.2222
Epoch 18/50
54/54 [==============================] - 1s 26ms/step - loss: 1.3678 - accuracy: 0.4444 - val_loss: 1.8111 - val_accuracy: 0.2778
Epoch 19/50
54/54 [==============================] - 1s 26ms/step - loss: 1.3675 - accuracy: 0.4259 - val_loss: 1.7913 - val_accuracy: 0.2222
Epoch 20/50
54/54 [==============================] - 1s 27ms/step - loss: 1.3191 - accuracy: 0.4074 - val_loss: 1.6433 - val_accuracy: 0.2222
Epoch 

Epoch 21/50
54/54 [==============================] - 1s 28ms/step - loss: 0.7393 - accuracy: 0.7222 - val_loss: 1.2817 - val_accuracy: 0.3889
Epoch 22/50
54/54 [==============================] - 1s 26ms/step - loss: 0.7772 - accuracy: 0.5556 - val_loss: 1.2338 - val_accuracy: 0.5556
Epoch 23/50
54/54 [==============================] - 1s 27ms/step - loss: 0.7164 - accuracy: 0.5556 - val_loss: 1.6930 - val_accuracy: 0.2778
Epoch 24/50
54/54 [==============================] - 1s 26ms/step - loss: 0.8322 - accuracy: 0.6481 - val_loss: 1.3082 - val_accuracy: 0.2778
Epoch 25/50
54/54 [==============================] - 1s 26ms/step - loss: 0.7213 - accuracy: 0.6481 - val_loss: 1.0799 - val_accuracy: 0.6111
Epoch 26/50
54/54 [==============================] - 1s 27ms/step - loss: 0.8177 - accuracy: 0.5741 - val_loss: 1.1905 - val_accuracy: 0.4444
Epoch 27/50
54/54 [==============================] - 1s 26ms/step - loss: 0.7155 - accuracy: 0.6667 - val_loss: 1.4839 - val_accuracy: 0.3333
Epoch 

In [27]:
avg = np.mean(results_list4, axis=0)
std_dev = np.std(results_list4, axis=0)
print('Average loss and accuracy scores were: ', avg)
print('Standard deviation for loss & accuracy: ', std_dev)
print('Duration in seconds: ', int(duration4))

Average loss and accuracy scores were:  [0.79460799 0.63333335]
Standard deviation for loss & accuracy:  [0.30347395 0.14529663]
Duration in seconds:  291


<h2> Expriment5:</h2>

In [28]:
net5 = models.Sequential()
net5.add(layers.Dense((480), input_shape = (480,6)))
net5.add(layers.Flatten())
net5.add(layers.Dense(240,activation='selu'))
net5.add(layers.Dense(240,activation='selu'))
net5.add(layers.Dense(240,activation='selu'))
net5.add(layers.Dense(7, activation = 'softmax'))
net5.summary()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_41 (Dense)             (None, 480, 480)          3360      
_________________________________________________________________
flatten_9 (Flatten)          (None, 230400)            0         
_________________________________________________________________
dense_42 (Dense)             (None, 240)               55296240  
_________________________________________________________________
dense_43 (Dense)             (None, 240)               57840     
_________________________________________________________________
dense_44 (Dense)             (None, 240)               57840     
_________________________________________________________________
dense_45 (Dense)             (None, 7)                 1687      
Total params: 55,416,967
T

In [29]:
net5.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
net5.save_weights('net5.h5')  # each split will start with same weights so can directly compare results

histories5 = []
results_list5 = []
index = 0
fold = val_split_size



start = tm.time()

for i in range (k):
    print(i)
    val_fold = train_data[index:fold]
    train_fold = np.delete(train_data, np.s_[index:fold],0)

    val_tgt = train_labels[index:fold]
    train_tgt = np.delete(train_labels, np.s_[index:fold])
    
    
    net5.load_weights('net5.h5')
    history = net5.fit(train_fold, train_tgt, epochs = 25, validation_data = (val_fold, val_tgt))

    histories5.append(history)
    
    
    results = net5.evaluate(test_data, test_labels)
    results_list5.append(results)
    print(results)
    
    index = index + val_split_size
    fold = fold + val_split_size

finish = tm.time()
duration5 = finish - start

0
Train on 54 samples, validate on 18 samples
Epoch 1/25
54/54 [==============================] - 2s 32ms/step - loss: 10.6962 - accuracy: 0.1111 - val_loss: 14.1889 - val_accuracy: 0.0556
Epoch 2/25
54/54 [==============================] - 1s 26ms/step - loss: 17.1072 - accuracy: 0.1667 - val_loss: 21.1174 - val_accuracy: 0.1111
Epoch 3/25
54/54 [==============================] - 1s 27ms/step - loss: 17.3418 - accuracy: 0.1667 - val_loss: 4.3712 - val_accuracy: 0.0556
Epoch 4/25
54/54 [==============================] - 1s 26ms/step - loss: 3.5426 - accuracy: 0.3519 - val_loss: 1.6168 - val_accuracy: 0.3333
Epoch 5/25
54/54 [==============================] - 1s 27ms/step - loss: 3.1897 - accuracy: 0.3889 - val_loss: 2.7532 - val_accuracy: 0.3333
Epoch 6/25
54/54 [==============================] - 1s 27ms/step - loss: 3.3622 - accuracy: 0.5556 - val_loss: 1.9738 - val_accuracy: 0.4444
Epoch 7/25
54/54 [==============================] - 1s 26ms/step - loss: 3.1886 - accuracy: 0.4815 - va

54/54 [==============================] - 1s 26ms/step - loss: 4.2666 - accuracy: 0.2963 - val_loss: 6.1993 - val_accuracy: 0.3889
Epoch 7/25
54/54 [==============================] - 1s 27ms/step - loss: 3.6333 - accuracy: 0.3889 - val_loss: 6.8976 - val_accuracy: 0.3333
Epoch 8/25
54/54 [==============================] - 1s 27ms/step - loss: 3.0102 - accuracy: 0.4815 - val_loss: 4.2417 - val_accuracy: 0.3333
Epoch 9/25
54/54 [==============================] - 1s 27ms/step - loss: 2.0634 - accuracy: 0.3704 - val_loss: 2.7578 - val_accuracy: 0.3333
Epoch 10/25
54/54 [==============================] - 1s 27ms/step - loss: 1.3087 - accuracy: 0.5741 - val_loss: 2.7640 - val_accuracy: 0.5000
Epoch 11/25
54/54 [==============================] - 1s 27ms/step - loss: 1.2524 - accuracy: 0.6296 - val_loss: 3.1801 - val_accuracy: 0.3889
Epoch 12/25
54/54 [==============================] - 1s 27ms/step - loss: 1.2324 - accuracy: 0.6296 - val_loss: 3.0716 - val_accuracy: 0.4444
Epoch 13/25
54/54 [==

In [30]:
avg = np.mean(results_list5, axis=0)
std_dev = np.std(results_list5, axis=0)
print('Average loss and accuracy scores were: ', avg)
print('Standard deviation for loss & accuracy: ', std_dev)
print('Duration in seconds: ', int(duration5))

Average loss and accuracy scores were:  [0.44806673 0.81666668]
Standard deviation for loss & accuracy:  [0.14883588 0.05527708]
Duration in seconds:  146
